### To Do / License





- For 5 users

- Input: Marketing campaign details

- Generate a custom image for a marketing campaign based upon their profile
  - Gemini needs to explain its reasoning
```
Generate a LLM Prompt to generate a marketing image to attact the attention to our Mocha Magic coffee drink based upon the customer's profile (in Json).
We are a coffee truck company called Chocolate A.I..
We want the image to be specific to this customer and their interests.
Think creatively and use the customer's interests to create a unique image.
Make sure you state the image should be photo realistic.
Avoid and copyrighted names such as sporting teams names.
{"children":"Yes","coffee_preferences":["French Press"],"content_interaction":["Social media engagement - high"],"customer_age":44,"customer_id":1,"customer_service_interactions":[],"dietary_preferences":[],"education":"Master of Fine Arts (MFA)","facebook_bio":"","facebook_handle":"","instagram_bio":"","instagram_handle":"","interests":["Photography","Ceramics","Pottery","Hiking"],"lifestyle":["Creative","Outdoorsy","Balanced"],"linkedin_bio":"Art educator passionate about fostering creativity and critical thinking skills in young minds. Experienced in curriculum development, classroom management, and arts integration. Seeking opportunities to collaborate with fellow educators and contribute to innovative art education initiatives.","linkedin_engagement":"Passive","linkedin_handle":"william-jones-art-educator","martial_status":"Married","occupation":"Art Teacher","solicated_buying_habits":["Special Occasions"],"sports":["San Francisco Giants"],"tiktok_bio":"","tiktok_handle":"","twitter_bio":"","twitter_engagement":"","twitter_handle":"","youtube_bio":"","youtube_engagement":"","youtube_handle":""}
```

- Pattern to show:
  1. Generate a basic prompt
  2. Have Gemini rewrite your prompt (better detail)
  3. Call Imagen3 with the updated prompt
  4. Pass the image back to Gemini Vision to verify the image meets the basic prompt

- Generate the marketing text

- Translate the text to different lanaguage
  1. Call Gemini to translate the text
  2. Pass the text back to Gemini to verify it is proper in that lanuage and is actually in the language


```
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

Author: Adam Paternostro

### Pip installs

In [ ]:
# To read/write to/from Kafka
import sys

# https://PLACEHOLDER.com/index.html
# !{sys.executable} -m pip install PLACEHOLDER

### Initialize

In [ ]:
from PIL import Image
from IPython.display import HTML
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random

In [ ]:
# Set these (run this cell to verify the output)

bigquery_location = "us"
region = "us-central1"
location = "us-central1"
storage_account = "data-analytics-preview"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

gemini_languages = ["Arabic (ar)",  "Bengali (bn)",  "Bulgarian (bg)",  "Chinese simplified (zh)",  "Chinese traditional (zh)",
  "Croatian (hr)",  "Czech (cs)",  "Danish (da)",  "Dutch (nl)",  "Estonian (et)",  "Finnish (fi)",  "French (fr)",
  "German (de)",  "Greek (el)",  "Hebrew (iw)",  "Hindi (hi)",  "Hungarian (hu)",  "Indonesian (id)",  "Italian (it)",  "Japanese (ja)",
  "Korean (ko)",  "Latvian (lv)",  "Lithuanian (lt)",  "Norwegian (no)",  "Polish (pl)",  "Portuguese (pt)",  "Romanian (ro)",  "Russian (ru)",
  "Serbian (sr)",  "Slovak (sk)",  "Slovenian (sl)",  "Spanish (es)",  "Swahili (sw)",  "Swedish (sv)",  "Thai (th)",  "Turkish (tr)",  "Ukrainian (uk)",
  "Vietnamese (vi)"]

# Get some values using gcloud
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### Helper Methods

#### restAPIHelper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### Imagen3 Image Generation

In [ ]:
def ImageGen(prompt):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  model_version = "imagen-3.0-generate-001" # imagen-3.0-fast-generate-001
  #model_version = "imagen-3.0-generate-preview-0611" # Preview Access Model

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/image-generation
  # url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project}/locations/{location}/publishers/google/models/imagegeneration:predict"
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project}/locations/{location}/publishers/google/models/{model_version}:predict"

  payload = {
    "instances": [
      {
        "prompt": prompt
      }
    ],
    "parameters": {
      "sampleCount": 1,
      "personGeneration" : "allow_adult"  # Google users will get an error due to an org policy
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    response_json = json.loads(response.content)
    print(f"Imagen3 response_json: {response_json}")

    if "blocked" in response_json:
      print(f"Blocked: {response_json['blocked']}")

    if "predictions" in response_json:
      image_data = response_json["predictions"][0]["bytesBase64Encoded"]
      image_data = base64.b64decode(image_data)
      filename= str(uuid.uuid4()) + ".png"
      with open(filename, "wb") as f:
        f.write(image_data)
      print(f"Image generated OK.")
      return filename
    else:
      raise RuntimeError(f"No predictions in response: {response.content}")
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

#### Gemini LLM (Pro 1.0 , Pro 1.5)

In [ ]:
def GeminiLLM(prompt, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

In [ ]:
def GeminiLLM_VerifyImage(prompt, imageBase64, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": [
          { "text": prompt },
          { "inlineData": {  "mimeType": "image/png", "data": f"{imageBase64}" } }
        ]
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Helper Functions

In [ ]:
def RunQuery(sql):
  import time
  from google.cloud import bigquery
  client = bigquery.Client()

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      raise Exception(query_job.error_result)

In [ ]:
def convert_png_to_base64(image_path):
  image = cv2.imread(image_path)

  # Convert the image to a base64 string.
  _, buffer = cv2.imencode('.png', image)
  base64_string = base64.b64encode(buffer).decode('utf-8')

  return base64_string

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)
  return json.dumps(json_object)

In [ ]:
# This was generated by GenAI

def copy_file_to_gcs(local_file_path, bucket_name, destination_blob_name):
  """Copies a file from a local drive to a GCS bucket.

  Args:
      local_file_path: The full path to the local file.
      bucket_name: The name of the GCS bucket to upload to.
      destination_blob_name: The desired name of the uploaded file in the bucket.

  Returns:
      None
  """

  import os
  from google.cloud import storage

  # Ensure the file exists locally
  if not os.path.exists(local_file_path):
      raise FileNotFoundError(f"Local file '{local_file_path}' not found.")

  # Create a storage client
  storage_client = storage.Client()

  # Get a reference to the bucket
  bucket = storage_client.bucket(bucket_name)

  # Create a blob object with the desired destination path
  blob = bucket.blob(destination_blob_name)

  # Upload the file from the local filesystem
  content_type = ""
  if local_file_path.endswith(".html"):
    content_type = "text/html; charset=utf-8"

  if local_file_path.endswith(".json"):
    content_type = "application/json; charset=utf-8"

  if content_type == "":
    blob.upload_from_filename(local_file_path)
  else:
    blob.upload_from_filename(local_file_path, content_type = content_type)

  print(f"File '{local_file_path}' uploaded to GCS bucket '{bucket_name}' as '{destination_blob_name}.  Content-Type: {content_type}'.")

### BigQuery Table and Preview Data

In [ ]:
%%bigquery

DROP TABLE IF EXISTS `chocolate_ai.customer_hyper_personalized_email`;

CREATE TABLE IF NOT EXISTS `chocolate_ai.customer_hyper_personalized_email`
(
    customer_id                                    INTEGER NOT NULL OPTIONS(description="Primary key."),

    llm_marketing_prompt                           STRING  OPTIONS(description="The prompt to generate the marketing email text."),
    llm_marketing_prompt_response_json             JSON    OPTIONS(description="The response from the llm marketing prompt in json."),
    llm_marketing_prompt_response_text             STRING  OPTIONS(description="The response from the llm marketing prompt in text."),

    llm_orginial_image_prompt                      STRING  OPTIONS(description="The prompt to generate the original image text."),
    llm_orginial_image_prompt_response_json        JSON    OPTIONS(description="The response from the llm original prompt in json."),
    llm_orginial_image_prompt_response_text        STRING  OPTIONS(description="The response from the llm original prompt in text."),

    llm_improved_image_prompt                      STRING  OPTIONS(description="The prompt to generate the improved image text."),
    -- The improved prompt will be passed to Imagen3
    --llm_improved_image_prompt_response_json      JSON    OPTIONS(description="The response from the llm improved prompt in json."),
    --llm_improved_image_prompt_response_text      STRING  OPTIONS(description="The response from the llm improved prompt in text."),

    llm_verify_image_prompt                        STRING  OPTIONS(description="The prompt to verify the generated image."),
    llm_verify_image_response_json                 JSON    OPTIONS(description="The response from verify the generated image in json."),
    llm_verify_image_text                          STRING  OPTIONS(description="The response from verify the generated image in text."),

    llm_translation_language_prompt                STRING  OPTIONS(description="The prompt to generate the secondary lanagage text."),
    llm_translation_language_prompt_response_json  JSON    OPTIONS(description="The response from the llm secondary lanagage prompt in json."),
    llm_translation_language_prompt_response_text  STRING  OPTIONS(description="The response from the llm secondary lanagage prompt in text."),

    llm_validate_translation_prompt                STRING  OPTIONS(description="The prompt to generate the vadiation of the translation text."),
    llm_validate_translation_prompt_response_json  JSON    OPTIONS(description="The response from the llm vadiation of the translation prompt in json."),
    llm_validate_translation_prompt_response_text  STRING  OPTIONS(description="The response from the llm vadiation of the translation prompt in text."),

    image_gcs_filename                             STRING  OPTIONS(description="The GCS path for the marketing campaign image."),
    image_http_url                                 STRING  OPTIONS(description="The HTTP path for the marketing campaign image."),
    image_generated                                BOOLEAN OPTIONS(description="Has the image been generated and saved to GCS."),
    image_verified                                 BOOLEAN OPTIONS(description="Did the image pass verification."),

    html_gcs_filename                              STRING  OPTIONS(description="The GCS path for the marketing campaign HTML file."),
    html_http_url                                  STRING  OPTIONS(description="The HTTP path for the marketing campaign HTML file."),
    html_generated                                 BOOLEAN OPTIONS(description="Has the HTML been generated and saved to GCS."),
    translation_verified                           BOOLEAN OPTIONS(description="Did the translation pass verification."),
)
CLUSTER BY customer_id;

In [ ]:
%%bigquery
SELECT *
  FROM `chocolate_ai.customer_marketing_profile`
 WHERE customer_profile_data        IS NOT NULL
   AND data_beans_profile_data      IS NOT NULL
   AND generated_marketing_insights IS NOT NULL
   AND customer_text_summary        IS NOT NULL;

### Get some customer process (you can adjust the limit clause)

In [ ]:
# Get the customers (if there are not any you can remove some from the chocolate_ai.customer_hyper_personalized_email table)
customer_list = []

sql = """WITH data AS (
  SELECT customer_marketing_profile.customer_id,
         customer_marketing_profile.customer_profile_data,
         customer_marketing_profile.data_beans_profile_data,
         customer_marketing_profile.generated_marketing_insights,
         customer_marketing_profile.customer_text_summary,
        JSON_VALUE(JSON_EXTRACT_ARRAY(data_beans_profile_data.top_3_favorite_menu_items,'$')[0]) as top_1_favorite_menu_item_id,
        JSON_VALUE(JSON_EXTRACT_ARRAY(data_beans_profile_data.top_3_favorite_menu_items,'$')[1]) as top_2_favorite_menu_item_id,
        JSON_VALUE(JSON_EXTRACT_ARRAY(data_beans_profile_data.top_3_favorite_menu_items,'$')[2]) as top_3_favorite_menu_item_id,
    FROM `chocolate_ai.customer_marketing_profile` AS customer_marketing_profile
  WHERE customer_profile_data        IS NOT NULL
    AND data_beans_profile_data      IS NOT NULL
    AND generated_marketing_insights IS NOT NULL
    AND customer_text_summary        IS NOT NULL
    AND NOT EXISTS (SELECT 1 FROM `chocolate_ai.customer_hyper_personalized_email` AS child where child.customer_id = customer_marketing_profile.customer_id)
  ORDER BY customer_id
  LIMIT 2
)
SELECT customer_id,
       customer_profile_data,
       data_beans_profile_data,
       generated_marketing_insights,
       customer_text_summary,
       CONCAT(menu_1.item_name,':',menu_1.llm_item_description) AS top_1_favorite_menu_item_name,
       CONCAT(menu_2.item_name,':',menu_1.llm_item_description) AS top_2_favorite_menu_item_name,
       CONCAT(menu_3.item_name,':',menu_1.llm_item_description) AS top_3_favorite_menu_item_name,
  FROM data
       INNER JOIN `chocolate_ai.menu` AS menu_1
               ON CAST(data.top_1_favorite_menu_item_id AS INT64) = menu_1.menu_id
       INNER JOIN `chocolate_ai.menu` AS menu_2
               ON CAST(data.top_2_favorite_menu_item_id AS INT64) = menu_2.menu_id
       INNER JOIN `chocolate_ai.menu` AS menu_3
               ON CAST(data.top_3_favorite_menu_item_id AS INT64) = menu_3.menu_id
"""

df_process = RunQuery(sql)

for row in df_process.itertuples():
  customer_dict = {
    "customer_id" : row.customer_id,
    "customer_profile_data" :row.customer_profile_data,
    "data_beans_profile_data" : row.data_beans_profile_data,
    "generated_marketing_insights" : row.generated_marketing_insights,
    "customer_text_summary" : row.customer_text_summary,
    "top_1_favorite_menu_item_name" : row.top_1_favorite_menu_item_name,
    "top_2_favorite_menu_item_name" : row.top_2_favorite_menu_item_name,
    "top_3_favorite_menu_item_name" : row.top_3_favorite_menu_item_name
  }
  print(f"Customer: {customer_dict}")
  customer_list.append(customer_dict)

# Just the Ids so I can query
customer_id_list = ([customer['customer_id'] for customer in customer_list])
customer_id_list_str = (', '.join(map(str, customer_id_list)))

### Generate and run the LLM Marketing Prompt

In [ ]:
# For each customer, generate the marketing prompt and run against Gemini

# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "customer_id" : 0,
#    "email_subject" : "text",
#    "marketing_text" : "text",
#    "explanation" : "text"
#  }
response_schema = {
  "type": "object",
  "required": [
    "customer_id",
    "email_subject",
    "marketing_text",
    "explanation"
  ],
  "properties": {
    "customer_id": {
      "type": "integer",
      "format": "int64"
    },
    "email_subject": {
      "type": "string"
    },
    "marketing_text": {
      "type": "string"
    },
    "explanation": {
      "type": "string"
    }
  }
}

for customer_dict in customer_list:
  print(f"Customer id: {customer_dict['customer_id']}")
  retry = 0
  success = False
  while not success:
    try:
      prompt = f"""You are a marketing expert and work at Chocolate A.I. a coffee truck company.
      You need to send out a hyper-personalized email to a customer.
      We to targeting their top favorite menu item(s) to be included in the email.
      Explain your reasoning and place that in the explanation field.
      Generate a marketing email for customer {customer_dict['customer_id']} using the below customer profile data.
        customer_profile_data: {customer_dict['customer_profile_data']}
        data_beans_profile_data: {customer_dict['data_beans_profile_data']}
        generated_marketing_insights: {customer_dict['generated_marketing_insights']}
        customer_text_summary: {customer_dict['customer_text_summary']}
        top_1_favorite_menu_item_name: {customer_dict['top_1_favorite_menu_item_name']}
        top_2_favorite_menu_item_name: {customer_dict['top_2_favorite_menu_item_name']}
        top_3_favorite_menu_item_name: {customer_dict['top_3_favorite_menu_item_name']}
        """

      print(prompt)
      llm_result = GeminiLLM(prompt,response_schema=response_schema)
      print(llm_result)
      json_result = json.loads(llm_result)
      result_escaped_quotes = llm_result.replace("\\","\\\\").replace("'","\'")
      #json_text = json_result['marketing_text'].replace("'","\'")

      # Save to database
      try:

        sql=f"""INSERT INTO `chocolate_ai.customer_hyper_personalized_email`
                            (customer_id, llm_marketing_prompt, llm_marketing_prompt_response_json, llm_marketing_prompt_response_text)
                    VALUES ({customer_dict['customer_id']}, \"\"\"{prompt}\"\"\", JSON\"\"\"{result_escaped_quotes}\"\"\",\"\"\"{json_result['marketing_text']}\"\"\")"""
        # print(sql)
        RunQuery(sql)

        # Jump out of loop
        customer_dict['marketing_text'] = json_result['marketing_text']
        customer_dict['email_subject'] = json_result['email_subject']
        success = True

        print("---------------------------------------------------------------------------------------")
        print(f"LLM Marketing Prompt [Success] for Customer {customer_dict['customer_id']}")
        print("---------------------------------------------------------------------------------------")

      except Exception as e:
        retry += 1
        print("---------------------------------------------------------------------------------------")
        print(f"LLM Marketing Prompt [SQL Error] for Customer {customer_dict['customer_id']}: {sql}")
        print("---------------------------------------------------------------------------------------")

    except Exception as e:
      retry += 1
      print("---------------------------------------------------------------------------------------")
      print(f"LLM Marketing Prompt [Error] for Customer {customer_dict['customer_id']}: {e}")
      print("---------------------------------------------------------------------------------------")

    if retry > 5:
      print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
      print(f"LLM Marketing Prompt [Retry Limit Reached - Skipping] for Customer {customer_dict['customer_id']}")
      print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
      break # Skip this customer

In [ ]:
# View Results
sql=f"""SELECT customer_id, llm_marketing_prompt, llm_marketing_prompt_response_json, llm_marketing_prompt_response_text
          FROM `chocolate_ai.customer_hyper_personalized_email`
         WHERE customer_id IN ({customer_id_list_str})"""

print(sql)
df_process = RunQuery(sql)
df_process.head()

### Create an image prompt and enhance it by running it through Gemini


In [ ]:
# For each customer, generate an improved image prompt using Gemini

# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "customer_id" : 0,
#    "image_prompt" : "text"
#    "explanation" : "text"
#  }
response_schema = {
  "type": "object",
  "required": [
    "customer_id",
    "image_prompt",
    "explanation"
  ],
  "properties": {
    "customer_id": {
      "type": "integer",
      "format": "int64"
    },
    "image_prompt": {
      "type": "string"
    },
    "explanation": {
      "type": "string"
    }
  }
}

for customer_dict in customer_list:
  print(f"Customer id: {customer_dict['customer_id']}")
  retry = 0
  success = False
  while not success:
    try:
      prompt = f"""You are a marketing expert and work at Chocolate A.I. a coffee truck company.
      You need to send out a hyper-personalized email to a customer.
      Generate a LLM Prompt to generate a marketing image based upon the customer's profile and the marketing message we are sending in the email.
      We are a coffee truck company called Chocolate A.I..
      We want the image to be specific to this customer and their interests.
      Think creatively and use the customer's interests to create a unique image.
      This is important to show something about their interests, hobbies, sports, etc.
      Do not include any names of professional sports teams since they are copyrighted.
      Make sure you state the image should be photo realistic.
      Avoid and copyrighted names such as sporting teams names.
      Avoid mentioning any celebrity names or specific people.
      Do not include references to kids or children in the image prompt.
      Only audits can be rendered by the image process.
      This this through step by step.
      Double check for kids, children, or copyrighted sports teams names.

      Customer's profile:
        customer_profile_data: {customer_dict['customer_profile_data']}
        data_beans_profile_data: {customer_dict['data_beans_profile_data']}
        generated_marketing_insights: {customer_dict['generated_marketing_insights']}
        customer_text_summary: {customer_dict['customer_text_summary']}
        top_1_favorite_menu_item_name: {customer_dict['top_1_favorite_menu_item_name']}
        top_2_favorite_menu_item_name: {customer_dict['top_2_favorite_menu_item_name']}
        top_3_favorite_menu_item_name: {customer_dict['top_3_favorite_menu_item_name']}

      Marketing Message:
       {customer_dict['marketing_text']}
        """

      print(prompt)
      llm_result = GeminiLLM(prompt,response_schema=response_schema)
      print(llm_result)
      json_result = json.loads(llm_result)
      result_escaped_quotes = llm_result.replace("\\","\\\\").replace("'","\'")

      # Save to database
      try:
        sql=f"""UPDATE `chocolate_ai.customer_hyper_personalized_email`
                   SET llm_orginial_image_prompt = \"\"\"{prompt}\"\"\",
                       llm_orginial_image_prompt_response_json = JSON\"\"\"{result_escaped_quotes}\"\"\",
                       llm_orginial_image_prompt_response_text = \"\"\"{json_result['image_prompt']}\"\"\",
                       llm_improved_image_prompt = \"\"\"{json_result['image_prompt']}\"\"\"
                 WHERE customer_id = {customer_dict['customer_id']}"""

        #print(sql)
        RunQuery(sql)

        # Jump out of loop
        customer_dict['image_prompt'] = json_result['image_prompt']
        customer_dict['image_explanation'] = json_result['explanation']
        success = True

        print("---------------------------------------------------------------------------------------")
        print(f"LLM Image Prompt [Success] for Customer {customer_dict['customer_id']}")
        print("---------------------------------------------------------------------------------------")

      except Exception as e:
        retry += 1
        print("---------------------------------------------------------------------------------------")
        print(f"LLM Image Prompt [SQL Error] for Customer {customer_dict['customer_id']}: {sql}")
        print("---------------------------------------------------------------------------------------")

    except Exception as e:
      retry += 1
      print("---------------------------------------------------------------------------------------")
      print(f"LLM Image Prompt [Error] for Customer {customer_dict['customer_id']}: {e}")
      print("---------------------------------------------------------------------------------------")

    if retry > 5:
      print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
      print(f"LLM Image Prompt [Retry Limit Reached - Skipping] for Customer {customer_dict['customer_id']}")
      print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
      break # Skip this customer

In [ ]:
# To view the bucket
print(f"https://console.cloud.google.com/storage/browser/{storage_account}/chocolate-ai/DB-GMA-Campaign-Assets-Hyper-Personalized-Email")

In [ ]:
# View Results
sql=f"""SELECT customer_id, llm_orginial_image_prompt, llm_orginial_image_prompt_response_json, llm_orginial_image_prompt_response_text, llm_improved_image_prompt
          FROM `chocolate_ai.customer_hyper_personalized_email`
         WHERE customer_id IN ({customer_id_list_str})"""

print(sql)
df_process = RunQuery(sql)
df_process.head()

### Call Imagen3 with the updated prompt


In [ ]:
# For each customer, generate the image using Imagen3
for customer_dict in customer_list:
  print(f"Customer id: {customer_dict['customer_id']}")
  try:
    image_prompt = customer_dict['image_prompt']
    print(f"Image Prompt: {image_prompt}")
    print(f"Image Prompt Explanation: {customer_dict['image_explanation']}")
    filename = ImageGen(customer_dict['image_prompt'])

    img = Image.open(filename)
    img.thumbnail([500,500]) # width, height
    IPython.display.display(img)

    # Save image to GCS
    dest_filename = f"email_campaign_{customer_dict['customer_id']}.png"
    copy_file_to_gcs(filename, storage_account, f"chocolate-ai/DB-GMA-Campaign-Assets-Hyper-Personalized-Email/email-{formatted_date}/{dest_filename}")
    customer_dict['gcs_filename'] = f"gs://{storage_account}/chocolate-ai/DB-GMA-Campaign-Assets-Hyper-Personalized-Email/email-{formatted_date}/{dest_filename}"
    customer_dict['html_filename'] = f"https://storage.cloud.google.com/{storage_account}/chocolate-ai/DB-GMA-Campaign-Assets-Hyper-Personalized-Email/email-{formatted_date}/{dest_filename}"

    # Update table in BigQuery
    try:
      sql=f"""UPDATE `chocolate_ai.customer_hyper_personalized_email`
                  SET image_gcs_filename = '{customer_dict['gcs_filename']}',
                      image_http_url = '{customer_dict['html_filename']}',
                      image_generated = TRUE
                WHERE customer_id = {customer_dict['customer_id']}"""

      #print(sql)
      RunQuery(sql)
      customer_dict['image_filename'] = filename

      print("---------------------------------------------------------------------------------------")
      print(f"Imagen3 [Success] for Customer {customer_dict['customer_id']}")
      print("---------------------------------------------------------------------------------------")

    except Exception as e:
      retry += 1
      print("---------------------------------------------------------------------------------------")
      print(f"Imagen3 [SQL Error] for Customer {customer_dict['customer_id']}: {sql}")
      print("---------------------------------------------------------------------------------------")

  except Exception as e:
    print("---------------------------------------------------------------------------------------")
    print(f"Imagen3 [Error] for Customer {customer_dict['customer_id']}: {e}")
    print("---------------------------------------------------------------------------------------")


In [ ]:
# View Results
sql=f"""SELECT customer_id, image_gcs_filename, image_http_url, image_generated
          FROM `chocolate_ai.customer_hyper_personalized_email`
         WHERE customer_id IN ({customer_id_list_str})"""

print(sql)
df_process = RunQuery(sql)
df_process.head()

### Verify the Generated Image with Gemini Vision

In [ ]:
# Verify the generate image is correct

# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "customer_id" : 0,
#    "image_verified" : true
#    "explanation" : "text"
#  }
response_schema = {
  "type": "object",
  "required": [
    "customer_id",
    "image_verified",
    "explanation"
  ],
  "properties": {
    "customer_id": {
      "type": "integer",
      "format": "int64"
    },
    "image_verified": {
      "type": "boolean"
    },
    "explanation": {
      "type": "string"
    }
  }
}

json_schema = '{ "image_verified" : true, "explanation" : "text" }'

for customer_dict in customer_list:
  print(f"Customer id: {customer_dict['customer_id']}")
  try:
    image_prompt = customer_dict['image_prompt']
    print(f"Image Prompt: {image_prompt}")
    print(f"Image Prompt Explanation: {customer_dict['image_explanation']}")
    filename = customer_dict['image_filename']

    prompt = f"""I need you to verify that the below image meets the following criteria:
    <ImagePrompt>
    {customer_dict['image_prompt']}
    </ImagePrompt>
    <ImageExplanation>
    {customer_dict['image_explanation']}
    </ImageExplanation>
    """

    print(prompt)
    imageBase64 = convert_png_to_base64(filename)
    llm_result = GeminiLLM_VerifyImage(prompt, imageBase64, response_schema=response_schema)
    print(llm_result)
    json_result = json.loads(llm_result)
    result_escaped_quotes = llm_result.replace("\\","\\\\").replace("'","\'")


    # Update table in BigQuery
    try:
      sql=f"""UPDATE `chocolate_ai.customer_hyper_personalized_email`
                  SET llm_verify_image_prompt = \"\"\"{prompt}\"\"\",
                      llm_verify_image_response_json = JSON\"\"\"{result_escaped_quotes}\"\"\",
                      llm_verify_image_text = \"\"\"{json_result['explanation']}\"\"\",
                      image_verified = {json_result['image_verified']}
                WHERE customer_id = {customer_dict['customer_id']}"""

      #print(sql)
      RunQuery(sql)

      print("---------------------------------------------------------------------------------------")
      print(f"Imagen3 Verification [Success] for Customer {customer_dict['customer_id']}")
      print("---------------------------------------------------------------------------------------")

    except Exception as e:
      retry += 1
      print("---------------------------------------------------------------------------------------")
      print(f"Imagen3 Verification [SQL Error] for Customer {customer_dict['customer_id']}: {sql}")
      print("---------------------------------------------------------------------------------------")

  except Exception as e:
    print("---------------------------------------------------------------------------------------")
    print(f"Imagen3 Verification [Error] for Customer {customer_dict['customer_id']}: {e}")
    print("---------------------------------------------------------------------------------------")


In [ ]:
# View Results
sql=f"""SELECT customer_id, llm_verify_image_prompt, llm_verify_image_response_json, llm_verify_image_text, image_verified
          FROM `chocolate_ai.customer_hyper_personalized_email`
         WHERE customer_id IN ({customer_id_list_str})"""

print(sql)
df_process = RunQuery(sql)
df_process.head()

### Translate the Marketing Message to another language

In [ ]:
# Translate the marketing text into another language (we will randomly pick on)

# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "customer_id" : 0,
#    "translated_text" : "text"
#  }
response_schema = {
  "type": "object",
  "required": [
    "customer_id",
    "translated_text"
  ],
  "properties": {
    "customer_id": {
      "type": "integer",
      "format": "int64"
    },
    "translated_text": {
      "type": "string"
    }
  }
}


for customer_dict in customer_list:
  # Pick an random language from the list
  random_language = random.randint(0,len(gemini_languages)-1)
  print(f"Random Language: {gemini_languages[random_language]}")
  print(f"Customer id: {customer_dict['customer_id']}")
  customer_dict['translation_language'] = gemini_languages[random_language]
  try:

    prompt = f"""You are an expert translator for the following language {gemini_languages[random_language]}.
    Translate the following text from English to {gemini_languages[random_language]}:
    <Text>
    {customer_dict['marketing_text']}
    </Text>
    """

    print(prompt)
    llm_result = GeminiLLM(prompt, response_schema=response_schema)
    print(llm_result)
    json_result = json.loads(llm_result)
    result_escaped_quotes = llm_result.replace("\\","\\\\").replace("'","\'")

    # Update table in BigQuery
    try:
      sql=f"""UPDATE `chocolate_ai.customer_hyper_personalized_email`
                  SET llm_translation_language_prompt = \"\"\"{prompt}\"\"\",
                      llm_translation_language_prompt_response_json = JSON\"\"\"{result_escaped_quotes}\"\"\",
                      llm_translation_language_prompt_response_text = \"\"\"{json_result['translated_text']}\"\"\"
                WHERE customer_id = {customer_dict['customer_id']}"""

      #print(sql)
      RunQuery(sql)
      customer_dict['translated_text'] = json_result['translated_text']

      print("---------------------------------------------------------------------------------------")
      print(f"Translation [Success] for Customer {customer_dict['customer_id']}")
      print("---------------------------------------------------------------------------------------")

    except Exception as e:
      retry += 1
      print("---------------------------------------------------------------------------------------")
      print(f"Translation [SQL Error] for Customer {customer_dict['customer_id']}: {sql}")
      print("---------------------------------------------------------------------------------------")

  except Exception as e:
    print("---------------------------------------------------------------------------------------")
    print(f"Translation [Error] for Customer {customer_dict['customer_id']}: {e}")
    print("---------------------------------------------------------------------------------------")


In [ ]:
# View Results
sql=f"""SELECT customer_id, llm_translation_language_prompt, llm_translation_language_prompt_response_json, llm_translation_language_prompt_response_text
          FROM `chocolate_ai.customer_hyper_personalized_email`
         WHERE customer_id IN ({customer_id_list_str})"""

print(sql)
df_process = RunQuery(sql)
df_process.head()

### Verify the Transalation

In [ ]:
# Verify the translation is correct

# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "customer_id" : 0,
#    "translation_verified" : true
#    "explanation" : "text"
#  }
response_schema = {
  "type": "object",
  "required": [
    "customer_id",
    "translation_verified",
    "explanation"
  ],
  "properties": {
    "customer_id": {
      "type": "integer",
      "format": "int64"
    },
    "translation_verified": {
      "type": "boolean"
    },
    "explanation": {
      "type": "string"
    }
  }
}


for customer_dict in customer_list:
  print(f"Customer id: {customer_dict['customer_id']}")
  try:
    prompt = f"""I need you to verify that the below text is in the langugage of "{customer_dict['translation_language']}".
    It was originially in English, so make sure it is not still English.
    <Text>
    {customer_dict['translated_text']}
    </Text>
    """

    print(prompt)
    llm_result = GeminiLLM(prompt, response_schema=response_schema)
    print(llm_result)
    json_result = json.loads(llm_result)
    result_escaped_quotes = llm_result.replace("\\","\\\\").replace("'","\'")

    # Update table in BigQuery
    try:
      sql=f"""UPDATE `chocolate_ai.customer_hyper_personalized_email`
                  SET llm_validate_translation_prompt = \"\"\"{prompt}\"\"\",
                      llm_validate_translation_prompt_response_json = JSON\"\"\"{result_escaped_quotes}\"\"\",
                      llm_validate_translation_prompt_response_text = \"\"\"{json_result['explanation']}\"\"\",
                      translation_verified = {json_result['translation_verified']}
                WHERE customer_id = {customer_dict['customer_id']}"""

      #print(sql)
      RunQuery(sql)

      print("---------------------------------------------------------------------------------------")
      print(f"Translation Verification [Success] for Customer {customer_dict['customer_id']}")
      print("---------------------------------------------------------------------------------------")

    except Exception as e:
      retry += 1
      print("---------------------------------------------------------------------------------------")
      print(f"Translation Verification [SQL Error] for Customer {customer_dict['customer_id']}: {sql}")
      print("---------------------------------------------------------------------------------------")

  except Exception as e:
    print("---------------------------------------------------------------------------------------")
    print(f"Translation Verification [Error] for Customer {customer_dict['customer_id']}: {e}")
    print("---------------------------------------------------------------------------------------")


In [ ]:
# View Results
sql=f"""SELECT customer_id, llm_validate_translation_prompt, llm_validate_translation_prompt_response_json, llm_validate_translation_prompt_response_text, translation_verified
          FROM `chocolate_ai.customer_hyper_personalized_email`
         WHERE customer_id IN ({customer_id_list_str})"""

print(sql)
df_process = RunQuery(sql)
df_process.head()

### Generate the HTML and Save

In [ ]:
html_template = """<!DOCTYPE html>
<html>
<head>
  <title>Coffee Campaign</title>
  <style>
    body {
      font-family: 'Helvetica Neue', sans-serif;
    }
    .email-campaign {
      background-color: #EDF2F9;
      padding: 20px;
      margin-bottom: 20px;
      border-bottom: 2px solid #ddd;
    }
    h3 {
      font-size: 16px;
      margin-bottom: 10px;
      color: #333;
    }
    p {
      font-size: 14px;
      line-height: 1.5;
      color: #555;
    }
  </style>
</head>
<body>
  <div class="email-campaign">
    <h3>Email Campaign (English)</h3>
    <p style="font-weight: bold;">Subject: ##email_subject##</p>
    <p>##marketing_text##</p>
  </div>
  <div>
    <img src="##html_filename##" width="500" height="500" alt="Item Image">
  </div>

  <hr/>

  <div class="email-campaign">
    <h3>Email Campaign (##translation_language##)</h3>
    <p>##translated_text##</p>
  </div>
  <div>
    <img src="##html_filename##" width="500" height="500" alt="Item Image">
  </div>

</body>
</html>
"""

In [ ]:
# Create HTML using the Template

for customer_dict in customer_list:
  if 'html_filename' not in customer_dict:
    # Error generating image
    print("Error: 'html_filename' not in customer_dict")
    continue

  if 'translated_text' not in customer_dict:
    # Error generating translation
    print("Error: 'translated_text' not in customer_dict")
    continue

  # Replace the placeholders with the actual values
  html = html_template \
    .replace("##email_subject##", customer_dict['email_subject']) \
    .replace("##marketing_text##", customer_dict['marketing_text']) \
    .replace("##translation_language##", customer_dict['translation_language']) \
    .replace("##translated_text##", customer_dict['translated_text']) \
    .replace("##html_filename##", customer_dict['html_filename'])

  filename = f"email_campaign_{customer_dict['customer_id']}.html"

  # Save the HTML to a file
  with open(filename, "w") as f:
    f.write(html)

  copy_file_to_gcs(filename, storage_account,f"chocolate-ai/DB-GMA-Campaign-Assets-Hyper-Personalized-Email/email-{formatted_date}/" + filename)

  # Update table in BigQuery
  try:
    html_gcs_filename = f"gs://{storage_account}/chocolate-ai/DB-GMA-Campaign-Assets-Hyper-Personalized-Email/email-{formatted_date}/{filename}"
    html_http_url = f"https://storage.cloud.google.com/{storage_account}/chocolate-ai/DB-GMA-Campaign-Assets-Hyper-Personalized-Email/email-{formatted_date}/{filename}"

    sql=f"""UPDATE `chocolate_ai.customer_hyper_personalized_email`
                SET html_gcs_filename = '{html_gcs_filename}',
                    html_http_url = '{html_http_url}',
                    html_generated = TRUE
              WHERE customer_id = {customer_dict['customer_id']}"""

    #print(sql)
    RunQuery(sql)

    print("---------------------------------------------------------------------------------------")
    print(f"HTML Generation [Success] for Customer {customer_dict['customer_id']}")
    print("---------------------------------------------------------------------------------------")

  except Exception as e:
    retry += 1
    print("---------------------------------------------------------------------------------------")
    print(f"HTML Generation [SQL Error] for Customer {customer_dict['customer_id']}: {sql}")
    print("---------------------------------------------------------------------------------------")


In [ ]:
# To view the bucket
print(f"https://console.cloud.google.com/storage/browser/{storage_account}/chocolate-ai/DB-GMA-Campaign-Assets-Hyper-Personalized-Email/email-{formatted_date}")

In [ ]:
# View Results
sql=f"""SELECT customer_id, html_gcs_filename, html_http_url, html_generated
          FROM `chocolate_ai.customer_hyper_personalized_email`
         WHERE customer_id IN ({customer_id_list_str})"""

print(sql)
df_process = RunQuery(sql)
df_process.head()

In [ ]:
filename = f"email_campaign_{customer_list[0]['customer_id']}.html"
IPython.display.HTML(filename=filename)

In [ ]:
filename = f"email_campaign_{customer_list[1]['customer_id']}.html"
IPython.display.HTML(filename=filename)

### View all results

In [ ]:
# View All Results
sql=f"""SELECT *
          FROM `chocolate_ai.customer_hyper_personalized_email`
         WHERE customer_id IN ({customer_id_list_str})"""

print(sql)
df_process = RunQuery(sql)
df_process.head()

### Clean Up

In [ ]:
# Placeholder

### Reference Links


- [Imagen3](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/image-generation)